# Murcielago System V1 - Training & testing

In [1]:
import torch
import torchaudio
from torch.utils.data import DataLoader
from custom_dataset import GunShotsNoisesDataset, split_dataset
import numpy as np
from cnn import ShotDetectionNetwork
from torch import nn
from train_test_functions import train_step, test_step, eval_model, accuracy_fn

## 1) Data preparing: Dataset and Dataloaders

In [2]:
metadata_file = "./dataset/metadata.xlsx"
audios_dir = "./dataset"
fs = 48000
scales = np.arange(1, 129)
transformation_dict = {"wavelet": "cmor","scales":scales}

GSN_visualize = GunShotsNoisesDataset(metadata_file, audios_dir, transformation_dict, fs, 0.01) #this returns audio waveform to analyse samples

GSN_dataset = GunShotsNoisesDataset(metadata_file, audios_dir, transformation_dict, fs)

In [3]:
batch_size = 4
train_data, test_data = split_dataset(GSN_dataset, 0.04)

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

## 2) Model building

In [4]:
mur_cnn_v1 = ShotDetectionNetwork(64)
print(mur_cnn_v1)

ShotDetectionNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): Conv2d(64, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 1, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=16, out_features=2, bias=True)
    (2): Softmax(dim=1)
  )
)


## 3) Train and Test

In [5]:
learn_rate = 0.01
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.SGD(mur_cnn_v1.parameters(), lr=learn_rate)

In [6]:
epochs = 20

for epoch in range(epochs):
    train_step(mur_cnn_v1, train_dataloader, loss_fn, optim, accuracy_fn=accuracy_fn)
    test_step(mur_cnn_v1, train_dataloader, loss_fn, accuracy_fn=accuracy_fn)
    

c:\Users\joaqo\anaconda3\envs\Sonido\Lib\site-packages\pywt\_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should takethe form cmorB-C where B and C are floats representing the bandwidth frequency and center frequency, respectively (example: cmor1.5-1.0).
  wavelet = DiscreteContinuousWavelet(wavelet)
c:\Users\joaqo\OneDrive\Escritorio\Programacion\Murcielago System\custom_dataset.py:147: UserWarning: ComplexHalf support is experimental and many operators don't support it yet. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\EmptyTensor.cpp:47.)
  coeffs = torch.tensor(coeffs, dtype=torch.complex32)


Train loss: 0.69601 | Train accuracy: 49.16667%
Test loss: 0.69335 | Test accuracy: 50.00000%
Train loss: 0.58799 | Train accuracy: 71.45833%
Test loss: 0.76115 | Test accuracy: 50.00000%
Train loss: 0.58226 | Train accuracy: 78.75000%
Test loss: 0.76212 | Test accuracy: 50.00000%
Train loss: 0.56242 | Train accuracy: 79.37500%
Test loss: 0.75635 | Test accuracy: 50.00000%
Train loss: 0.54875 | Train accuracy: 78.95833%
Test loss: 0.73961 | Test accuracy: 50.00000%
Train loss: 0.51995 | Train accuracy: 81.45833%
Test loss: 0.73485 | Test accuracy: 50.00000%
Train loss: 0.49210 | Train accuracy: 83.54167%
Test loss: 0.73669 | Test accuracy: 50.00000%
Train loss: 0.47309 | Train accuracy: 84.58333%
Test loss: 0.73867 | Test accuracy: 50.00000%
Train loss: 0.43135 | Train accuracy: 88.75000%
Test loss: 0.76125 | Test accuracy: 50.00000%
Train loss: 0.42640 | Train accuracy: 89.37500%
Test loss: 0.75900 | Test accuracy: 50.00000%
Train loss: 0.43208 | Train accuracy: 88.75000%
Test loss: 0

## 4) Results